# Data Mart Design
## Creating Fact and Dimension Views using Star Schema

| **Name**             | **Master Table**                 | **Joined With**                                | **Notes**                                                                                                   |
|-----------------------|---------------------------------|-------------------------------------------------|--------------------------------------------------------------------------------------------------------------|
| **gold.dim_customers** | silver.crm_cust_info            | silver.erp_cust_az12, silver.erp_loc_a101      | - Checked for duplicates in master table's potential primary key that might be introduced after the join.     |
|                       |                                 |                                                 | - Fixed data integration issue for gender by prioritizing master table, using secondary table if unavailable. |
|                       |                                 |                                                 | - Renamed and rearranged the columns in a more meaningful way.                                                |
|                       |                                 |                                                 | - Generated a surrogate key with window function **ROW_NUMBER()** to ensure uniqueness.                        |
| **gold.dim_products**  | silver.crm_prd_info             | silver.erp_px_cat_g1v2                         | - Checked for duplicates in master table's potential primary key that might be introduced after the join.     |
|                       |                                 |                                                 | - Historical data is filtered out.                                                                            |
|                       |                                 |                                                 | - Renamed and rearranged the columns in a more meaningful way.                                                |
|                       |                                 |                                                 | - Generated a surrogate key with window function **ROW_NUMBER()** to ensure uniqueness.                        |
| **gold.fact_sales**    | silver.crm_sales_details        | gold.dim_products, gold.dim_customers          | - Performed data lookup for dimensions' surrogate keys for easier connection with dimensions.                  |
|                       |                                 |                                                 | - Renamed and rearranged the columns in a more meaningful way.                                                |
|                       |                                 |                                                 | - Checked the data model connectivity between fact and dimensions.                                             |


## Dim: Customers

In [2]:
-- Joining the two tables
-- checking the result in the newly added columns (bdate, gen, cntry)
SELECT TOP 10
        ci.cst_id, 
        ci.cst_key, 
        ci.cst_firstname, 
        ci.cst_lastname, 
        ci.cst_marital_status, 
        ci.cst_gndr, 
        ci.cst_create_date, 
        ca.bdate, 
        ca.gen, 
        la.cntry
    FROM silver.crm_cust_info ci
    LEFT JOIN silver.erp_cust_az12 ca 
        ON ci.cst_key = ca.cid
    LEFT JOIN silver.erp_loc_a101 la 
        ON ci.cst_key = la.cid

(20 rows affected)

Total execution time: 00:00:00.048

cst_id,cst_key,cst_firstname,cst_lastname,cst_marital_status,cst_gndr,cst_create_date,bdate,gen,cntry
11010,AW00011010,Jacquelyn,Suarez,Single,Female,2025-10-06,1969-08-05,Female,Australia
11011,AW00011011,Curtis,Lu,Married,Male,2025-10-06,1969-05-03,Male,Australia
11012,AW00011012,Lauren,Walker,Married,Female,2025-10-06,1979-01-14,Female,United States
11013,AW00011013,Ian,Jenkins,Married,Male,2025-10-06,1979-08-03,Male,United States
11027,AW00011027,Jessie,Zhao,Married,Male,2025-10-07,1952-06-05,Female,Australia
11028,AW00011028,Jill,Jimenez,Married,Female,2025-10-07,1951-10-09,Female,Australia
11029,AW00011029,Jimmy,Moreno,Married,Male,2025-10-07,1952-06-19,Female,Australia
11030,AW00011030,Bethany,Yuan,Married,Female,2025-10-07,1958-02-18,Female,Australia
11043,AW00011043,Nathan,Simmons,Married,Male,2025-10-07,1981-08-23,Female,United States
11044,AW00011044,Adam,Flores,Married,Male,2025-10-07,1954-11-21,Female,Australia


In [3]:
-- Checking for duplicates in master table's potenital primary key that might be introduced after the join 
-- (might happen if the other tables have duplicates in their columns used for join)
-- we need to do this step as we need it for later join with fact sales view
-- Fortunately, second table did not cause duplicates

SELECT 
    cst_id, 
    COUNT(*) AS cnt
FROM (
    SELECT 
        ci.cst_id, 
        ci.cst_key, 
        ci.cst_firstname, 
        ci.cst_lastname, 
        ci.cst_marital_status, 
        ci.cst_gndr, 
        ci.cst_create_date, 
        ca.bdate, 
        ca.gen, 
        la.cntry
    FROM silver.crm_cust_info ci
    LEFT JOIN silver.erp_cust_az12 ca 
        ON ci.cst_key = ca.cid
    LEFT JOIN silver.erp_loc_a101 la 
        ON ci.cst_key = la.cid
) AS t
GROUP BY cst_id
HAVING COUNT(*) > 1
OR cst_id IS NULL;

(0 rows affected)

Total execution time: 00:00:00.085

cst_id,cnt


In [8]:
-- gender data integration issue
SELECT DISTINCT 
    ci.cst_gndr,
    ca.gen
FROM silver.crm_cust_info AS ci
LEFT JOIN silver.erp_cust_az12 AS ca
    ON ci.cst_key = ca.cid
LEFT JOIN silver.erp_loc_a101 AS la
    ON ci.cst_key = la.cid
ORDER BY 1,2;

(9 rows affected)

Total execution time: 00:00:00.051

cst_gndr,gen
Female,Female
Female,Male
Female,Unknown
Male,Female
Male,Male
Male,Unknown
Unknown,Female
Unknown,Male
Unknown,Unknown


In [12]:
-- As the CRM is the master table. we can use it to fix the conflicted gender cases. 
-- If it is Unknown in CRM table, we use second table

SELECT DISTINCT 
    ci.cst_gndr,
    ca.gen,
    CASE 
        WHEN ci.cst_gndr != 'Unknown' THEN ci.cst_gndr -- CRM is the Master for gender Info
        ELSE COALESCE(ca.gen, 'Unknown')
    END AS new_gen
FROM silver.crm_cust_info AS ci
LEFT JOIN silver.erp_cust_az12 AS ca
    ON ci.cst_key = ca.cid
    ORDER BY 1,2;

(9 rows affected)

Total execution time: 00:00:00.070

cst_gndr,gen,new_gen
Female,Female,Female
Female,Male,Female
Female,Unknown,Female
Male,Female,Male
Male,Male,Male
Male,Unknown,Male
Unknown,Female,Female
Unknown,Male,Male
Unknown,Unknown,Unknown


In [14]:
-- Now it's time to rename and rearrange the columns in a more meaningful way
-- Also adding a surrogate key to ensure that it is always unique and causes no issue for data modeling later on
SELECT TOP 5
    ROW_NUMBER() OVER (ORDER BY cst_id) AS customer_key, -- Surrogate key
    ci.cst_id                          AS customer_id,
    ci.cst_key                         AS customer_number,
    ci.cst_firstname                   AS first_name,
    ci.cst_lastname                    AS last_name,
    la.cntry                           AS country,
    ci.cst_marital_status              AS marital_status,
    CASE 
        WHEN ci.cst_gndr != 'Unknown' THEN ci.cst_gndr -- CRM is the primary source for gender
        ELSE COALESCE(ca.gen, 'Unknown')  			   -- Fallback to ERP data
    END                                AS gender,
    ca.bdate                           AS birthdate,
    ci.cst_create_date                 AS create_date
FROM silver.crm_cust_info ci
LEFT JOIN silver.erp_cust_az12 ca
    ON ci.cst_key = ca.cid
LEFT JOIN silver.erp_loc_a101 la
    ON ci.cst_key = la.cid;

(5 rows affected)

Total execution time: 00:00:00.109

customer_key,customer_id,customer_number,first_name,last_name,country,marital_status,gender,birthdate,create_date
1,11000,AW00011000,Jon,Yang,Australia,Married,Male,1971-10-06,2025-10-06
2,11001,AW00011001,Eugene,Huang,Australia,Single,Male,1976-05-10,2025-10-06
3,11002,AW00011002,Ruben,Torres,Australia,Married,Male,1971-02-09,2025-10-06
4,11003,AW00011003,Christy,Zhu,Australia,Single,Female,1973-08-14,2025-10-06
5,11004,AW00011004,Elizabeth,Johnson,Australia,Single,Female,1979-08-05,2025-10-06


## Dim: Products

In [16]:
-- As we remember, the product info table conains current and historical data
-- Now we need to decide to stick with historical or current data
-- I will fillter our historical data

SELECT TOP 10
    pn.prd_id, 
    pn.cat_id, 
    pn.prd_key, 
    pn.prd_nm, 
    pn.prd_cost, 
    pn.prd_line, 
    pn.prd_start_dt
FROM silver.crm_prd_info AS pn
WHERE pn.prd_end_dt IS NULL; -- Filter out all historical data

(10 rows affected)

Total execution time: 00:00:00.008

prd_id,cat_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt
478,AC_BC,BC-M005,Mountain Bottle Cage,4,Mountain,2013-07-01
479,AC_BC,BC-R205,Road Bottle Cage,3,Road,2013-07-01
477,AC_BC,WB-H098,Water Bottle - 30 oz.,2,Other Sales,2013-07-01
483,AC_BR,RA-H123,Hitch Rack - 4-Bike,45,Other Sales,2013-07-01
486,AC_BS,ST-1401,All-Purpose Bike Stand,59,Mountain,2013-07-01
484,AC_CL,CL-9009,Bike Wash - Dissolver,3,Other Sales,2013-07-01
485,AC_FE,FE-6654,Fender Set - Mountain,8,Mountain,2013-07-01
217,AC_HE,HL-U509,Sport-100 Helmet- Black,13,Other Sales,2013-07-01
222,AC_HE,HL-U509-B,Sport-100 Helmet- Blue,13,Other Sales,2013-07-01
214,AC_HE,HL-U509-R,Sport-100 Helmet- Red,13,Other Sales,2013-07-01


In [17]:
-- Joining with silver.erp_px_cat_g1v2
SELECT TOP 10
    pn.prd_id, 
    pn.cat_id, 
    pn.prd_key, 
    pn.prd_nm, 
    pn.prd_cost, 
    pn.prd_line, 
    pn.prd_start_dt, 
    pc.cat, 
    pc.subcat, 
    pc.maintenance
FROM silver.crm_prd_info AS pn
LEFT JOIN silver.erp_px_cat_g1v2 AS pc
    ON pn.cat_id = pc.id
WHERE pn.prd_end_dt IS NULL; -- Filter out all historical data

(10 rows affected)

Total execution time: 00:00:00.032

prd_id,cat_id,prd_key,prd_nm,prd_cost,prd_line,prd_start_dt,cat,subcat,maintenance
478,AC_BC,BC-M005,Mountain Bottle Cage,4,Mountain,2013-07-01,Accessories,Bottles and Cages,No
479,AC_BC,BC-R205,Road Bottle Cage,3,Road,2013-07-01,Accessories,Bottles and Cages,No
477,AC_BC,WB-H098,Water Bottle - 30 oz.,2,Other Sales,2013-07-01,Accessories,Bottles and Cages,No
483,AC_BR,RA-H123,Hitch Rack - 4-Bike,45,Other Sales,2013-07-01,Accessories,Bike Racks,Yes
486,AC_BS,ST-1401,All-Purpose Bike Stand,59,Mountain,2013-07-01,Accessories,Bike Stands,No
484,AC_CL,CL-9009,Bike Wash - Dissolver,3,Other Sales,2013-07-01,Accessories,Cleaners,Yes
485,AC_FE,FE-6654,Fender Set - Mountain,8,Mountain,2013-07-01,Accessories,Fenders,No
217,AC_HE,HL-U509,Sport-100 Helmet- Black,13,Other Sales,2013-07-01,Accessories,Helmets,Yes
222,AC_HE,HL-U509-B,Sport-100 Helmet- Blue,13,Other Sales,2013-07-01,Accessories,Helmets,Yes
214,AC_HE,HL-U509-R,Sport-100 Helmet- Red,13,Other Sales,2013-07-01,Accessories,Helmets,Yes


In [18]:
-- Checking the uniqueness of prd_key as we need this for later join with fact sales view
-- Fortunately, second table did not cause duplicates
SELECT prd_key, COUNT(*) FROM (
    SELECT 
    pn.prd_id, 
    pn.cat_id, 
    pn.prd_key, 
    pn.prd_nm, 
    pn.prd_cost, 
    pn.prd_line, 
    pn.prd_start_dt, 
    pc.cat, 
    pc.subcat, 
    pc.maintenance
FROM silver.crm_prd_info AS pn
LEFT JOIN silver.erp_px_cat_g1v2 AS pc
    ON pn.cat_id = pc.id
WHERE pn.prd_end_dt IS NULL
)t GROUP BY prd_key HAVING COUNT (*) > 1

(0 rows affected)

Total execution time: 00:00:00.052

prd_key,(No column name)


In [19]:
-- Now it's time to rename and rearrange the columns in a more meaningful way
-- Also adding a surrogate key to ensure that it is always unique and causes no issue for data modeling later on
SELECT TOP 5
    ROW_NUMBER() OVER (ORDER BY pn.prd_start_dt, pn.prd_key) AS product_key, -- Surrogate key
    pn.prd_id       AS product_id,
    pn.prd_key      AS product_number,
    pn.prd_nm       AS product_name,
    pn.cat_id       AS category_id,
    pc.cat          AS category,
    pc.subcat       AS subcategory,
    pc.maintenance  AS maintenance,
    pn.prd_cost     AS cost,
    pn.prd_line     AS product_line,
    pn.prd_start_dt AS start_date
FROM silver.crm_prd_info pn
LEFT JOIN silver.erp_px_cat_g1v2 pc
    ON pn.cat_id = pc.id
WHERE pn.prd_end_dt IS NULL; -- Filter out all historical data

(5 rows affected)

Total execution time: 00:00:00.025

product_key,product_id,product_number,product_name,category_id,category,subcategory,maintenance,cost,product_line,start_date
1,210,FR-R92B-58,HL Road Frame - Black- 58,CO_RF,Components,Road Frames,Yes,0,Road,2003-07-01
2,211,FR-R92R-58,HL Road Frame - Red- 58,CO_RF,Components,Road Frames,Yes,0,Road,2003-07-01
3,348,BK-M82B-38,Mountain-100 Black- 38,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01
4,349,BK-M82B-42,Mountain-100 Black- 42,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01
5,350,BK-M82B-44,Mountain-100 Black- 44,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01


## Fact: Sales

In [21]:
-- Use the dimension's surrogate keys instead of IDs to easily connect facts with dimensions
-- Data lookup for dimensions' surrogate keys
SELECT TOP 5
    sd.sls_ord_num,
    pr.product_key,
    cu.customer_key,
    sd.sls_order_dt,
    sd.sls_ship_dt,
    sd.sls_due_dt,
    sd.sls_sales,
    sd.sls_quantity,
    sd.sls_price
FROM silver.crm_sales_details AS sd
LEFT JOIN gold.dim_products AS pr
    ON sd.sls_prd_key = pr.product_number
LEFT JOIN gold.dim_customers AS cu
    ON sd.sls_cust_id = cu.customer_id;

(5 rows affected)

Total execution time: 00:00:00.098

sls_ord_num,product_key,customer_key,sls_order_dt,sls_ship_dt,sls_due_dt,sls_sales,sls_quantity,sls_price
SO43697,20,10769,2010-12-29,2011-01-05,2011-01-10,3578,1,3578
SO43698,9,17390,2010-12-29,2011-01-05,2011-01-10,3400,1,3400
SO43699,9,14864,2010-12-29,2011-01-05,2011-01-10,3400,1,3400
SO43700,41,3502,2010-12-29,2011-01-05,2011-01-10,699,1,699
SO43701,9,4,2010-12-29,2011-01-05,2011-01-10,3400,1,3400


In [22]:
-- Now it's time to rename and rearrange the columns in a more meaningful way
SELECT TOP 5
    sd.sls_ord_num  AS order_number, -- keys
    pr.product_key  AS product_key,
    cu.customer_key AS customer_key,
    sd.sls_order_dt AS order_date,    -- dates
    sd.sls_ship_dt  AS shipping_date,
    sd.sls_due_dt   AS due_date,
    sd.sls_sales    AS sales_amount,  -- measures
    sd.sls_quantity AS quantity,
    sd.sls_price    AS price
FROM silver.crm_sales_details sd
LEFT JOIN gold.dim_products pr
    ON sd.sls_prd_key = pr.product_number
LEFT JOIN gold.dim_customers cu
    ON sd.sls_cust_id = cu.customer_id;

(5 rows affected)

Total execution time: 00:00:00.101

order_number,product_key,customer_key,order_date,shipping_date,due_date,sales_amount,quantity,price
SO43697,20,10769,2010-12-29,2011-01-05,2011-01-10,3578,1,3578
SO43698,9,17390,2010-12-29,2011-01-05,2011-01-10,3400,1,3400
SO43699,9,14864,2010-12-29,2011-01-05,2011-01-10,3400,1,3400
SO43700,41,3502,2010-12-29,2011-01-05,2011-01-10,699,1,699
SO43701,9,4,2010-12-29,2011-01-05,2011-01-10,3400,1,3400


In [23]:
-- Check the data model connectivity between fact and dimensions
SELECT * 
FROM gold.fact_sales f
LEFT JOIN gold.dim_customers c
ON c.customer_key = f.customer_key
LEFT JOIN gold.dim_products p
ON p.product_key = f.product_key
WHERE p.product_key IS NULL OR c.customer_key IS NULL  


(0 rows affected)

Total execution time: 00:00:00.228

order_number,product_key,customer_key,order_date,shipping_date,due_date,sales_amount,quantity,price,customer_key,customer_id,customer_number,first_name,last_name,country,marital_status,gender,birthdate,create_date,product_key,product_id,product_number,product_name,category_id,category,subcategory,maintenance,cost,product_line,start_date
